## Word2Vec ## 

In this notebook we will go through the step by step creation of the Continouous Bag Of Words (CBOW).
CBOW is an embedded model that makes use of a "fake task" -> [within short window, predict the current word] to extract a vector that shows the relationship between words.

### Continuous Bag Of Words ###

Adapted from Robert Guthrie

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.linalg

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# CBOW is a window view; we are trying to infer the word in the middle.
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

raw_text= """Long Short-Term Memory (LSTM) is a recurrent neural network (RNN) architecture that has been designed
to address the vanishing and exploding gradient problems of conventional RNNs. Unlike feedforward neural networks,
RNNs have cyclic connections making them powerful for modeling sequences. 
They have been successfully used for sequence labeling and sequence prediction tasks,
such as handwriting recognition, language modeling, phonetic labeling of acoustic frames. However, in contrast to the deep neural
networks, the use of RNNs in speech recognition has been limited to phone recognition in small scale tasks. 
In this paper, we present novel LSTM based RNN architectures which make more effective
use of model parameters to train acoustic models for large vocabulary speech recognition. 
We train and compare LSTM, RNN and DNN models at various numbers of parameters and configurations.
We show that LSTM models converge quickly and give state of the art speech recognition performance for relatively small sized models.""".split()

# By deriving a set from "raw_text", we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

# Basic Tokenizer
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(raw_text))
print(vocab_size)

152
106


In [4]:
# Now lets create a "dataset"
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = []
    for j in range(CONTEXT_SIZE, 0, -1):
        context.append(raw_text[i - j])

    for j in range(1, CONTEXT_SIZE + 1):
        context.append(raw_text[i + j])
        
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


[(['Long', 'Short-Term', '(LSTM)', 'is'], 'Memory'), (['Short-Term', 'Memory', 'is', 'a'], '(LSTM)'), (['Memory', '(LSTM)', 'a', 'recurrent'], 'is'), (['(LSTM)', 'is', 'recurrent', 'neural'], 'a'), (['is', 'a', 'neural', 'network'], 'recurrent')]


### Create the CBOW Model (as we have seen already other ANN) ###

We have to extend from nn.Module as all the other ANN

In [5]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embed_dim, context, hidden_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(context*embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim = -1)
        )
        
    def forward(self, inputs):
#         print(inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
#         print(out.shape)
        out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
#         print(out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out


#### Lets break it down! ####

In [6]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

example_tensor = torch.randint(0, VOCAB_SIZE, [BATCH_SIZE, FULL_CONTEXT_SIZE])
print(example_tensor)
print(example_tensor.shape)

tensor([[ 90,  61,  33,  49],
        [ 58,  78,  39,  73],
        [ 28,  89, 100,  38],
        [ 96,  87, 105,  70],
        [ 73,  75,  66,  97],
        [ 21,  70,  11,  86]])
torch.Size([6, 4])


In [7]:
CBOW_embedding = nn.Embedding(VOCAB_SIZE, EMBEDD_DIM)
example_result = CBOW_embedding(example_tensor)

# Now we have a representation of the words in a vector of EMBEDD_DIM Dimensions
print(example_result.shape) # [batch_size, context_size*2, EMBEDD_DIM]

# example_result = torch.flatten(example_result, start_dim=1)
example_result = example_result.view(BATCH_SIZE, -1)
print(example_result.shape)

torch.Size([6, 4, 10])
torch.Size([6, 40])


In [8]:
CBOW_hidden = nn.Linear(EMBEDD_DIM * FULL_CONTEXT_SIZE, HIDDEN_SIZE)
CBOW_hidden_relu = nn.ReLU()

example_result = CBOW_hidden(example_result)
example_result = CBOW_hidden_relu(example_result)
print(example_result.shape)

torch.Size([6, 256])


In [9]:
CBOW_output = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE)
CBOW_output_soft = nn.LogSoftmax(dim = -1)

example_result = CBOW_output(example_result)
example_result = CBOW_output_soft(example_result)
print(example_result.shape) # 106 (vocab_size) -> [0.002, 0.003, 0.02, ... , n] see which one of the elements are the highest

torch.Size([6, 106])


In [10]:
print(example_result[0].argmax(-1))
print(example_result[0])
print(example_result[1].argmax(-1))
print(example_result[1])
# print(example_result[2].argmax(-1))
# print(example_result[3].argmax(-1))
# print(example_result[4].argmax(-1))

tensor(62)
tensor([-4.9167, -4.5047, -4.7207, -4.5055, -4.7720, -4.9722, -4.4946, -4.4996,
        -4.7299, -4.4009, -4.7111, -4.6857, -4.5435, -4.6608, -4.5856, -4.4767,
        -4.5991, -4.6841, -4.6008, -4.9535, -4.9594, -4.4287, -4.6698, -4.6422,
        -4.7063, -4.8353, -4.8744, -4.5088, -4.7658, -4.9647, -4.5921, -4.8477,
        -4.7816, -4.4679, -4.7227, -4.6913, -4.5758, -4.6915, -4.4834, -4.4173,
        -4.5685, -4.4338, -4.3896, -4.5655, -4.3284, -4.4830, -4.8171, -4.8943,
        -4.5705, -4.6621, -4.5704, -5.0100, -4.9647, -4.9311, -4.6785, -4.9392,
        -4.6119, -4.6453, -4.5574, -4.3053, -4.8308, -4.6025, -4.1148, -4.5481,
        -4.8202, -4.7501, -4.7595, -4.7200, -5.3668, -4.5206, -4.7111, -4.7795,
        -4.8865, -4.5948, -4.7522, -4.8628, -4.8054, -4.4269, -4.4656, -4.8375,
        -4.4048, -4.9503, -5.0272, -4.6859, -4.8072, -4.7599, -4.7577, -4.3968,
        -4.8185, -4.3154, -4.6443, -4.5852, -4.4690, -4.9610, -4.6297, -4.9639,
        -4.7057, -4.6115, -4.

## Back to the notebook ##

In [11]:
# Simple helper method to transform the context to the expected int vector - tensor

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)

tensor([94, 44, 40, 14])

In [12]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for context, target in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector(context, word_to_ix)
            context_idxs = context_idxs.to(device)
            
            target_id = make_context_vector([target], word_to_ix)
            target_id = target_id.to(device)

            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #         loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [13]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 10
BATCH_SIZE = 6
FULL_CONTEXT_SIZE = CONTEXT_SIZE * 2
HIDDEN_SIZE = 256

loss_function = nn.NLLLoss() # Because we are using Log_softmax
model = CBOW(vocab_size, EMBEDD_DIM, FULL_CONTEXT_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # The loss decreased every iteration over the training data!

[697.5052881240845, 690.5906434059143, 683.7699465751648, 677.0398046970367, 670.3956201076508, 663.8321220874786, 657.342515707016, 650.9206223487854, 644.5612609386444, 638.2598602771759, 632.0147347450256, 625.8205578327179, 619.6723718643188, 613.5701248645782, 607.5069546699524, 601.4806604385376, 595.4874603748322, 589.5206079483032, 583.5748262405396, 577.6465163230896, 571.732962846756, 565.8326027393341, 559.9430418014526, 554.0597102642059, 548.1808083057404, 542.3050093650818, 536.4271016120911, 530.5495131015778, 524.6662369966507, 518.776873588562, 512.8807660341263, 506.97223019599915, 501.05758571624756, 495.13486552238464, 489.19853377342224, 483.2521538734436, 477.2936267852783, 471.32160913944244, 465.3371968269348, 459.34034979343414, 453.33001387119293, 447.30980610847473, 441.2786830663681, 435.2417069673538, 429.19513869285583, 423.1394194364548, 417.0780557990074, 411.01127177476883, 404.9383143186569, 398.86509931087494, 392.7885857820511, 386.71150213479996, 38

In [14]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())

In [15]:
def similarity_cbow(word_1, word_2):
    
    # test word similarity
    print(word_1)
    print(word_2)
    w1_id = torch.tensor(word_to_ix[word_1], dtype=torch.long)
    w2_id = torch.tensor(word_to_ix[word_2], dtype=torch.long)
    w1_id = w1_id.to(device)
    w2_id = w2_id.to(device)
    
    word_1_vec = model.get_word_vector(w1_id)
    word_2_vec = model.get_word_vector(w2_id)
    
    # The norm of a vector (1D-matrix) is the square root of the sum of all the squared values within the vector.
    print(math.sqrt(torch.square(word_1_vec).sum()))    
    print(torch.linalg.norm(word_1_vec))
    print(torch.linalg.norm(word_2_vec))
    print(word_1_vec.dot(word_2_vec))
    
    word_distance = torch.linalg.norm(word_1_vec - word_2_vec)
    print("Distance between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_distance))
    word_similarity = (word_1_vec.dot(word_2_vec) / (torch.linalg.norm(word_1_vec) * torch.linalg.norm(word_2_vec)))
    print("Similarity between '{}' & '{}' : {:0.4f}".format(word_1, word_2, word_similarity))


In [16]:
similarity_cbow("neural", "network")

neural
network
3.527445676562381
tensor(3.5274, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(3.7175, device='cuda:0', grad_fn=<CopyBackwards>)
tensor(-5.5670, device='cuda:0', grad_fn=<DotBackward>)
Distance between 'neural' & 'network' : 6.1153
Similarity between 'neural' & 'network' : -0.4245


In [17]:
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

In [18]:
predict_middle_word("a recurrent", "network is")
predict_middle_word("LSTM is", "recurrent neural")

neural
a


### Now that you saw how to create the CBOW model (word2vec), you should work on doing the "opposite" model, Skip-Gram ###

Skip-gram as you saw on the lectures, reverses the problem so you need to predict through the "fake task" the context of the input

# Lab assignment below

In [19]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embed_dim, context,target_size, hidden_size):
        super(Skipgram, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, vocab_size*target_size),
            nn.LogSoftmax(dim = 1)
        )
        self.target_size=target_size
        
    def forward(self, inputs):
      #  print("0", inputs.shape)
#         print(inputs)
        out = self.embedding(inputs)
     #   print("1", out.shape)
        #print(out.shape)
        #out = out.view(1, -1)
#         print(out.shape)
        out = self.linear(out)
      #  print("2", out.shape)
        out = out.view(self.target_size, -1)
      #  print("3", out.shape)
        return out
    
    # This is what we are actually interested on
    def get_word_vector(self, word):
        out = self.embedding(word)
        return out

In [20]:
def train(model, epochs, data, optimizer, loss_fn):
    model.train()
    losses = []
    for epoch in range(epochs):
        total_loss = 0
        for target, context in data:

            # Prepare inputs and targets 
            context_idxs = make_context_vector([context], word_to_ix)
            context_idxs = context_idxs.to(device)

            target_id = make_context_vector(target, word_to_ix)
            target_id = target_id.to(device)
            
            # Do not accumulate 
            model.zero_grad()

            # Step 3. Run the forward pass
            log_probs = model(context_idxs)
    #         break

            # Step 4. Compute your loss function.
            loss = loss_fn(log_probs, target_id)

    #       loss = loss_function(log_probs, torch.tensor([word_to_ix[context]], dtype=torch.long))

            # Step 5. Do the backward pass and update the gradient
            loss.backward()
            optimizer.step()

            # Get the Python number from a 1-element Tensor by calling tensor.item()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
    

In [21]:
VOCAB_SIZE = len(vocab)
EMBEDD_DIM = 106
CONTEXT_SIZE = 1
HIDDEN_SIZE = 512 # 256
TARGET_SIZE = 4

loss_function = nn.NLLLoss() # Because we are using 
model = Skipgram(vocab_size, EMBEDD_DIM, CONTEXT_SIZE, TARGET_SIZE, HIDDEN_SIZE)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = train(model, 100, data, optimizer, loss_function)
model.eval()

print(losses)  # Loss decreasing over time (iterations of training)

[903.1108045578003, 899.5648446083069, 896.043426990509, 892.5458483695984, 889.0705842971802, 885.6167793273926, 882.1825251579285, 878.7671947479248, 875.36900806427, 871.9868407249451, 868.6202774047852, 865.2679481506348, 861.9296927452087, 858.6043934822083, 855.291042804718, 851.9882702827454, 848.6961522102356, 845.4132289886475, 842.1381502151489, 838.8706531524658, 835.6094164848328, 832.3545713424683, 829.1052265167236, 825.8613700866699, 822.6209464073181, 819.3850607872009, 816.1520094871521, 812.9225950241089, 809.6945934295654, 806.468111038208, 803.243161201477, 800.0170922279358, 796.7909359931946, 793.564197063446, 790.3363180160522, 787.1080965995789, 783.8784775733948, 780.6464247703552, 777.4115467071533, 774.1744046211243, 770.9334411621094, 767.6897888183594, 764.4427080154419, 761.1906213760376, 757.9347906112671, 754.6747288703918, 751.4101343154907, 748.1406664848328, 744.8672037124634, 741.5882654190063, 738.3047499656677, 735.0147638320923, 731.7201466560364,

In [22]:
#import matplotlib.pyplot as plt
#plt.figure()
#plt.plot(losses)

In [23]:
# list out keys and values separately
key_list = list(word_to_ix.keys())
val_list = list(word_to_ix.values())
print(key_list)
print(val_list)

['Memory', 'based', 'neural', 'at', 'numbers', 'architecture', 'we', 'They', 'language', 'speech', 'connections', 'powerful', 'a', 'have', 'is', 'sized', 'contrast', 'deep', 'small', 'has', 'quickly', 'paper,', 'network', 'RNNs.', 'sequence', 'modeling,', 'frames.', 'models.', 'as', 'address', 'use', 'RNNs', 'cyclic', 'In', 'feedforward', 'vocabulary', 'of', 'for', 'acoustic', 'RNN', '(LSTM)', 'art', 'phonetic', 'However,', 'Short-Term', 'the', 'relatively', 'them', 'Unlike', 'configurations.', 'tasks,', 'various', 'problems', 'recognition', 'to', 'successfully', 'architectures', 'conventional', 'state', 'LSTM,', 'prediction', 'show', 'model', 'give', 'exploding', 'that', 'recurrent', 'DNN', 'which', 'limited', 'used', 'handwriting', 'in', 'large', 'making', 'parameters', 'phone', 'converge', 'novel', 'make', 'tasks.', 'networks,', 'LSTM', 'sequences.', 'effective', 'performance', 'labeling', 'present', 'vanishing', 'compare', 'train', 'models', 'more', 'recognition,', 'Long', 'recogni

Reverse cbow and use the middle word to predict the 2 words adjacent to the middle word.

In [24]:
"""
def predict_middle_word(prev_words, post_words):
    prev_words = prev_words.split()
    post_words = post_words.split()

    input_words= make_context_vector(prev_words + post_words, word_to_ix)
    input_words = input_words.to(device)
    output = model(input_words)
    out_ind = output.argmax(1)
#     print(word_to_ix)
#     out_word = word_to_ix.itos[out_ind.item()]
    out_word = key_list[val_list.index(out_ind.item())]
    print(out_word)

"""

# Reverse cbow and use the middle word to predict the 2 words adjacent to the middle word.
def predict_adjacent(middle_word):
    context_words = []
    
    print("Word context:", middle_word)
    input_words = make_context_vector([middle_word], word_to_ix)
    input_words = input_words.to(device)
    #output = model.get_word_vector(input_words)
    output = model(input_words)
    prediction = output.argmax(1)
   # pred = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html 
    #print("predictions", prediction)
    print("oi", prediction)  
    
    #for word in prediction[1][0]:
    for word in prediction:
        print("word tensor:", word)
        context_words.append(key_list[word])
       # print("test", context_words)
    context_words.insert(int(len(context_words)/2), middle_word)
    print("\n", context_words)



In [25]:
predict_adjacent("neural")

Word context: neural
oi tensor([45, 17, 81, 45], device='cuda:0')
word tensor: tensor(45, device='cuda:0')
word tensor: tensor(17, device='cuda:0')
word tensor: tensor(81, device='cuda:0')
word tensor: tensor(45, device='cuda:0')

 ['the', 'deep', 'neural', 'networks,', 'the']


In [26]:
"""   
def predict_topk(middle_word):
    print("Word context:", middle_word)
    input_words = make_context_vector([middle_word], word_to_ix)
    input_words = input_words.to(device)
    output = model.get_word_vector(input_words)
    #out_index = output.argmax(1)
    
    prediction = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html 
    print("predictions", prediction)
  
    context_words=[]
    for word in prediction[1][0]:
        print("word tensor", word)
        context_words.append(key_list[word])
       # print("test", context_words)
    context_words.insert(int(len(context_words)/2), middle_word)
    print(context_words)
"""


'   \ndef predict_topk(middle_word):\n    print("Word context:", middle_word)\n    input_words = make_context_vector([middle_word], word_to_ix)\n    input_words = input_words.to(device)\n    output = model.get_word_vector(input_words)\n    #out_index = output.argmax(1)\n    \n    prediction = torch.topk(output, 4) # return the most likely elements/words. https://pytorch.org/docs/stable/generated/torch.topk.html \n    print("predictions", prediction)\n  \n    context_words=[]\n    for word in prediction[1][0]:\n        print("word tensor", word)\n        context_words.append(key_list[word])\n       # print("test", context_words)\n    context_words.insert(int(len(context_words)/2), middle_word)\n    print(context_words)\n'

In [27]:
#predict_topk("neural")

In [28]:
#similarity_cbow("neural", "network")